In [1]:
import requests 
from bs4 import BeautifulSoup as soup
import pandas as pd


In [3]:
#requesting news response from bbc
bbc_news_url = "https://www.bbc.co.uk/news"
response = requests.get(bbc_news_url)
response.status_code


200

In [4]:
#Soup the response
bbc_soup = soup(response.content, 'lxml')
article_div = bbc_soup.find_all( attrs={"class": "gs-c-promo-body gel-1/2@xs gel-1/1@m"})

#extracting article details
head_line=[]
article_url=[]
article_summary=[]
article_date_time=[]

for article in article_div:
    try:
        title = article.find('h3').text
        print(title)
        head_line.append(title)
    except:
        print("exception while extracting head line")
    
    try:
        href = article.find('a', href=True)
        url = "https://www.bbc.com"+href['href']
        print(url)
        article_url.append(url)
    except:
        print("exception while extracting url")
        
    try:
        summary = article.find('p').text
        print(summary)
        article_summary.append(summary)
    except:
        print("exception while extracting summary")
        
    try:
        date_time = article_div[0].find('time')['datetime']
        print(date_time)
        article_date_time.append(date_time)
    except:
        print("exception while extracting date time")
            


'You are not alone' says Queen in Christmas speech
https://www.bbc.com/news/uk-55447011
She says that while all many people want is "a squeeze of the hand", there is "hope in the new dawn".
2020-12-25T15:50:19.000Z
UK minister sees 'special relationship' with EU
https://www.bbc.com/news/uk-politics-55451102
The cabinet secretary says he has lost friends over Brexit and admits it turned UK politics "ugly".
2020-12-25T15:50:19.000Z
UN peacekeepers killed in CAR on election eve
https://www.bbc.com/news/world-africa-55451763
The UN said three peacekeepers died in two separate attacks, as rebel and government forces clash.
2020-12-25T15:50:19.000Z
Australian saved in five-day Antarctica rescue
https://www.bbc.com/news/world-australia-55446454
Australia, China and the US work together to medically evacuate the expeditioner.
2020-12-25T15:50:19.000Z
Millions wake up to tighter Covid restrictions in UK
https://www.bbc.com/news/uk-55449596
More areas in England enter tier four, as lockdowns beg

In [5]:
#dataframe the article details
data = {'head_line': head_line, 'url': article_url, 'summary':article_summary, 'date': article_date_time}
df = pd.DataFrame.from_dict(data)
df.head(1)

,head_line,url,summary,date
0,'You are not alone' says Queen in Christmas sp...,https://www.bbc.com/news/uk-55447011,"She says that while all many people want is ""a...",2020-12-25T15:50:19.000Z


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   head_line  10 non-null     object
 1   url        10 non-null     object
 2   summary    10 non-null     object
 3   date       10 non-null     object
dtypes: object(4)
memory usage: 448.0+ bytes


In [7]:
#function to get response from url nad soup it
#it takes a list of url's and return a list of souped responses
def get_article(article_url):

    article_soup=[]
    for url in article_url:
        try:
            response = requests.get(url)
            article_soup.append(soup(response.content, 'lxml'))
        except:
            print("error fetching response")
        
    return(article_soup)

#get list of soup
article_soup = get_article(article_url)
#get list of article text blocks
text_blocks = article_soup[0].find_all(attrs={"data-component": "text-block"})
#append all text blocks in one text and append it in article text list
article_text=[]
text=""
for article in article_soup:
    try:
        text_blocks = article.find_all(attrs={"data-component": "text-block"})
    except:
        print("error finding blocks")
    for block in text_blocks:
        try:
            text += block.find('p').text+"\n"
        except:
            print("error extracting text")
        
    article_text.append(text)
    text = ""
    text_blocks=[]

In [8]:
#creat a column for article text in df
df['text']=article_text
df.head(1)

,head_line,url,summary,date,text
0,'You are not alone' says Queen in Christmas sp...,https://www.bbc.com/news/uk-55447011,"She says that while all many people want is ""a...",2020-12-25T15:50:19.000Z,The Queen has used her Christmas Day message t...


In [9]:
#save the df into csv file
df.to_csv("bbc_scraped_articles.csv", index=False)